# Submission Preparation

In [1]:
import pandas as pd
import os

SYNTH_DATA_PATH = '../data'
OUT_SYNTH_DATA_PATH = '../output'
SOLUTION_FILE = 'solution_yearly_test.csv'

- We have provided a sample solution.csv file for your reference. Your submission must
follow the same format.
- If you don’t provide values for all valid indices, a default value i.e. zero will be considered
as the value for those indices. This may result into constraint violation.
- For data_type depot_location and refinery_location, entries under destination_index and
value are not required and will be disregarded. You may choose to keep these entries blank.
- For data_type biomass_forecast, entries under destination_index are not required and will be
disregarded. You may choose to keep these entries blank.
- Optimized supply chain infrastructure proposed in your solution must be the same for both
year 2018 and 2019. Consequently, the entries for data_type depot_location and
refinery_location will be agnostic to year. Use year = 20182019 for such common entries.
- You can only place one depot per grid block/location. Similarly, You can only place one
biorefinery per grid block/location.
- Your solution will be eligible for ranking only if it satisfies all the constraints for 2018 and 2019.
- You do not need to submit your source code files. When you submit your solution, you can
ignore the "Upload source code" field.
- We will keep the first year (2018) of your solution for the public leaderboard. You can test
your solution any time and see how it ranks.
- We will keep the second year (2019) of your solution for the private leaderboard and it will
be used to determine the finalists.

In [4]:
df_submission = pd.read_csv(os.path.join(SYNTH_DATA_PATH, 'sample_submission.csv'))
df_submission.head()

,year,data_type,source_index,destination_index,value
0,20182019,depot_location,1256,NaN,NaN
1,20182019,depot_location,1595,NaN,NaN
2,20182019,depot_location,1271,NaN,NaN
3,20182019,depot_location,2001,NaN,NaN
4,20182019,depot_location,2201,NaN,NaN


In [ ]:
df_submission['data_type'].value_counts()

data_type
biomass_demand_supply    21646
biomass_forecast          4836
pellet_demand_supply       152
depot_location              21
refinery_location            4
Name: count, dtype: int64

In [30]:
df_sol = pd.read_csv(os.path.join(OUT_SYNTH_DATA_PATH, SOLUTION_FILE))
df_sol.columns = ["data_type", "solution"]
df_sol.head()

,data_type,solution
0,b_2018_0_0,7.387925
1,b_2018_0_1,0.000000
2,b_2018_0_2,0.000000
3,b_2018_0_3,0.000000
4,b_2018_0_4,0.000000


In [37]:
df_fc = pd.read_csv(os.path.join(SYNTH_DATA_PATH, 'Biomass_History_Synthetic.csv'))
df_fc.head()

,Index,Latitude,Longitude,2018,2019
0,0,24.66818,71.33144,7.387925,5.180296
1,1,24.66818,71.41106,40.431847,42.126945
2,2,24.66818,71.49069,59.181629,73.203232
3,3,24.66818,71.57031,74.536720,101.067352
4,4,24.66818,71.64994,16.531315,26.086885


In [53]:
df_fc_sol = df_fc.copy()
df_fc_sol = df_fc_sol.melt(value_vars=['2018', '2019'], var_name='year', value_name='biomass', id_vars=['Index'])
df_fc_sol['data_type'] = 'biomass_forecast'
df_fc_sol['destination_index'] = None
df_fc_sol = df_fc_sol[['data_type', 'year', 'Index', 'destination_index', 'biomass']]
df_fc_sol.columns = ['data_type', 'year', 'source_index', 'destination_index', 'value']
df_fc_sol.head()

,data_type,year,source_index,destination_index,value
0,biomass_forecast,2018,0,None,7.387925
1,biomass_forecast,2018,1,None,40.431847
2,biomass_forecast,2018,2,None,59.181629
3,biomass_forecast,2018,3,None,74.536720
4,biomass_forecast,2018,4,None,16.531315


## Biomass

In [67]:
df_sol_proc = df_sol.copy()
df_sol_proc['data_type'] = df_sol_proc['data_type'].str.replace('x_', 'x_20182019_').str.replace('r_', 'r_20182019_')
df_sol_proc = df_sol_proc['data_type'].str.split("_", expand=True)
df_sol_proc.columns = ['data_type', 'year', 'source_index', 'destination_index']

df_sol_proc['value'] = df_sol['solution']
df_sol_proc['data_type'] = df_sol_proc['data_type'].map({'b': 'biomass_demand_supply', 
                                                         'p': 'pellet_demand_supply', 
                                                         'x': 'depot_location', 
                                                         'r': 'refinery_location'})
df_sol_proc = df_sol_proc[df_sol_proc['value'] != 0]

df_sol_proc = pd.concat([df_sol_proc, df_fc_sol])
df_sol_proc.head()

,data_type,year,source_index,destination_index,value
0,biomass_demand_supply,2018,0,0,7.387925
22,biomass_demand_supply,2018,1,1,40.431847
44,biomass_demand_supply,2018,2,2,59.181629
66,biomass_demand_supply,2018,3,3,74.536720
88,biomass_demand_supply,2018,4,4,16.531315


In [68]:
df_sol_proc[df_sol_proc['data_type'] == 'depot_location'].head()

,data_type,year,source_index,destination_index,value
203112,depot_location,20182019,0,None,1.0
203113,depot_location,20182019,1,None,1.0
203114,depot_location,20182019,2,None,1.0
203115,depot_location,20182019,3,None,1.0
203116,depot_location,20182019,4,None,1.0


In [69]:
df_sol_proc[df_sol_proc['data_type'] == 'refinery_location'].head()

,data_type,year,source_index,destination_index,value
203133,refinery_location,20182019,0,None,1.0
203134,refinery_location,20182019,1,None,1.0
203135,refinery_location,20182019,2,None,1.0
203136,refinery_location,20182019,3,None,1.0


In [70]:
df_sol_proc.data_type.value_counts()

data_type
biomass_forecast         4836
biomass_demand_supply    3589
depot_location             21
pellet_demand_supply        7
refinery_location           4
Name: count, dtype: int64

In [71]:
df_sol_proc[df_sol_proc['data_type'] == 'biomass_demand_supply'].head()

,data_type,year,source_index,destination_index,value
0,biomass_demand_supply,2018,0,0,7.387925
22,biomass_demand_supply,2018,1,1,40.431847
44,biomass_demand_supply,2018,2,2,59.181629
66,biomass_demand_supply,2018,3,3,74.536720
88,biomass_demand_supply,2018,4,4,16.531315


In [72]:
df_sol_proc[df_sol_proc['data_type'] == 'pellet_demand_supply'].head()

,data_type,year,source_index,destination_index,value
101578,pellet_demand_supply,2018,1,1,18759.985496
101600,pellet_demand_supply,2018,2,2,100000.000000
101622,pellet_demand_supply,2018,3,3,100000.000000
152334,pellet_demand_supply,2019,0,0,100000.000000
152356,pellet_demand_supply,2019,1,1,100000.000000


In [ ]:
df_sol_proc[df_sol_proc['data_type'] == 'pellet_demand_supply'].head()

In [73]:
df_sol_proc.to_csv(os.path.join(OUT_SYNTH_DATA_PATH, 'subm_yearly_test.csv'), index=False)